In [1]:
# import librosa
# import os
import numpy as np
# import matplotlib.pyplot as plt
import pandas as pd
# import IPython.display as ipd
# from sklearn import preprocessing 
# import winsound
import torch
import torch.nn as nn
# import torch.optim as optim
# from torch.autograd import Variable
import torch.nn.functional as F
# from torch.utils.data import Dataset
# from collections import Counter
from tqdm import tqdm
# from sklearn.metrics import classification_report 
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# spectrogram_array = np.loadtxt('spectrograms_Atrain.csv', delimiter=',')
# spectrogram_array = spectrogram_array.reshape(32400, 256, 74) # reshaping the array to have time frames as 3 dimensions

In [3]:
mfcc_array_2d = np.loadtxt('/kaggle/input/dataset/mfcc_train.csv', delimiter=',')
mfcc_array = mfcc_array_2d.reshape(len(mfcc_array_2d), 20, 74)

In [4]:
# spectrograms_tensor_Atrain = torch.tensor(spectrogram_array, dtype=torch.float32)
# spectrograms_tensor_Atrain = spectrograms_tensor_Atrain.unsqueeze(1)

In [5]:
# spectrograms_tensor_Atrain.shape

In [4]:
training_tensor = torch.tensor(mfcc_array, dtype=torch.float32)
training_tensor = training_tensor.unsqueeze(1)

In [5]:
output_data = pd.read_csv('/kaggle/input/dataset/train.csv', header = None)

/tmp/ipykernel_34/1821392965.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  output_data = pd.read_csv('/kaggle/input/dataset/train.csv', header = None)


In [6]:
output_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,ID,model_type_A,model_type_B,model_type_C,maneuvering_direction_B,maneuvering_direction_C,maneuvering_direction_CC,maneuvering_direction_F,maneuvering_direction_L,maneuvering_direction_R,fault_MF1,fault_MF2,fault_MF3,fault_MF4,fault_N,fault_PC1,fault_PC2,fault_PC3,fault_PC4
1,A_B_MF1_mic1_0_ConstructionSite_527_snr=12.519...,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,A_B_MF1_mic1_100_EoeunHill_279_snr=11.12957698...,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,A_B_MF1_mic1_101_WestDoor_321_snr=12.345834132...,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,A_B_MF1_mic1_103_ConstructionSite_465_snr=11.8...,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97196,C_R_PC4_mic1_974_ConstructionSite_720_snr=12.8...,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
97197,C_R_PC4_mic1_975_DuckPond_142_snr=12.327612455...,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
97198,C_R_PC4_mic1_977_SportsComplex_566_snr=13.2281...,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
97199,C_R_PC4_mic1_97_WestDoor_280_snr=14.8309085957...,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [7]:
maneuvering_direction = output_data[[4, 5, 6, 7, 8, 9]].values[1:]
maneuvering_direction = maneuvering_direction.astype('int32')
fault = output_data[[10, 11, 12, 13, 14, 15, 16, 17,18]].values[1:]
fault = fault.astype('int32')
model_type = output_data[[1, 2, 3]].values[1:]
model_type = model_type.astype('int32')

In [8]:
y1 = torch.tensor(model_type, dtype=torch.float16).cuda()
y2 = torch.tensor(maneuvering_direction, dtype=torch.float16).cuda()
y3 = torch.tensor(fault, dtype=torch.float16).cuda()
Xtr = torch.tensor(training_tensor, dtype=torch.float32).cuda()

/tmp/ipykernel_34/1052091417.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xtr = torch.tensor(training_tensor, dtype=torch.float32).cuda()


In [11]:
# Xtrain = torch.tensor(spectrograms_tensor_Atrain, dtype=torch.float32).cuda()

In [9]:
dataset = TensorDataset(Xtr, y1, y2, y3)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [10]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.softplus1 = nn.Softplus(beta=1, threshold=20) 
        self.softplus2 = nn.Softplus(beta=1, threshold=20)
        self.relu = nn.ReLU(inplace=True)
        

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = self.softplus1(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        shortcut = self.shortcut(x)
        out += shortcut
        out = self.softplus2(out)  
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.rl = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.rl(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        return out

class MultiTaskResNet18(nn.Module):
    def __init__(self, num_classes1=3, num_classes2=6, num_classes3=9):
        super(MultiTaskResNet18, self).__init__()
        self.resnet18 = ResNet(BasicBlock, [3, 4, 6, 3])
        self.fc1 = nn.Linear(512, num_classes1)
        self.fc2 = nn.Linear(512, num_classes2)
        self.fc3 = nn.Linear(512, num_classes3)
        self.sm1 = nn.Softmax(dim=1)#dim=1)
        self.sm2 = nn.Softmax(dim=1)#dim=1)
        self.sm3 = nn.Softmax(dim=1)#dim=1)

    def forward(self, x):
        out = self.resnet18(x)
        out1 = self.sm1(self.fc1(out))#, dim=1)
        out2 = self.sm2(self.fc2(out))#, dim=1)
        out3 = self.sm3(self.fc3(out))#, dim=1)
        return out1, out2, out3

model = MultiTaskResNet18(num_classes1=3, num_classes2=6, num_classes3=9).cuda()

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)

In [ ]:
num_epochs = 100
for epoch in tqdm(range(num_epochs)):
    for batch_idx, (data, target1, target2, target3) in enumerate(dataloader):
        data, target1, target2, target3 = data.cuda(), target1.cuda(), target2.cuda(), target3.cuda()
        outputs1, outputs2, outputs3 = model(data)

        loss1 = criterion(outputs1, target1)
        loss2 = criterion(outputs2, target2)
        loss3 = criterion(outputs3, target3)
        total_loss = loss1 + loss2 + loss3
    
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        # scheduler.step()

        if (batch_idx+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(dataloader)}], Total Loss: {total_loss.item():.4f}')
# winsound.Beep(440, 500)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/100], Step [10/760], Total Loss: 4.8308
Epoch [1/100], Step [20/760], Total Loss: 4.7618
Epoch [1/100], Step [30/760], Total Loss: 4.6770
Epoch [1/100], Step [40/760], Total Loss: 4.5977
Epoch [1/100], Step [50/760], Total Loss: 4.6209
Epoch [1/100], Step [60/760], Total Loss: 4.5580
Epoch [1/100], Step [70/760], Total Loss: 4.5208
Epoch [1/100], Step [80/760], Total Loss: 4.5350
Epoch [1/100], Step [90/760], Total Loss: 4.5219
Epoch [1/100], Step [100/760], Total Loss: 4.4654
Epoch [1/100], Step [110/760], Total Loss: 4.5502
Epoch [1/100], Step [120/760], Total Loss: 4.5031
Epoch [1/100], Step [130/760], Total Loss: 4.4577
Epoch [1/100], Step [140/760], Total Loss: 4.5011
Epoch [1/100], Step [150/760], Total Loss: 4.4438
Epoch [1/100], Step [160/760], Total Loss: 4.4451
Epoch [1/100], Step [170/760], Total Loss: 4.5107
Epoch [1/100], Step [180/760], Total Loss: 4.4828
Epoch [1/100], Step [190/760], Total Loss: 4.5462
Epoch [1/100], Step [200/760], Total Loss: 4.5136
Epoch [1/

  1%|          | 1/100 [02:21<3:53:12, 141.34s/it]

Epoch [1/100], Step [760/760], Total Loss: 4.1599
Epoch [2/100], Step [10/760], Total Loss: 4.2404
Epoch [2/100], Step [20/760], Total Loss: 4.2083
Epoch [2/100], Step [30/760], Total Loss: 4.0684
Epoch [2/100], Step [40/760], Total Loss: 4.0677
Epoch [2/100], Step [50/760], Total Loss: 4.1473
Epoch [2/100], Step [60/760], Total Loss: 4.1022
Epoch [2/100], Step [70/760], Total Loss: 4.1603
Epoch [2/100], Step [80/760], Total Loss: 4.2534
Epoch [2/100], Step [90/760], Total Loss: 4.1650
Epoch [2/100], Step [100/760], Total Loss: 4.1143
Epoch [2/100], Step [110/760], Total Loss: 4.0847
Epoch [2/100], Step [120/760], Total Loss: 4.1750
Epoch [2/100], Step [130/760], Total Loss: 4.1325
Epoch [2/100], Step [140/760], Total Loss: 4.2004
Epoch [2/100], Step [150/760], Total Loss: 4.1997
Epoch [2/100], Step [160/760], Total Loss: 4.1348
Epoch [2/100], Step [170/760], Total Loss: 4.1769
Epoch [2/100], Step [180/760], Total Loss: 4.1071
Epoch [2/100], Step [190/760], Total Loss: 4.1206
Epoch [2/

  2%|▏         | 2/100 [04:41<3:49:42, 140.64s/it]

...

Epoch [65/100], Step [760/760], Total Loss: 3.1073
Epoch [66/100], Step [10/760], Total Loss: 3.0875
Epoch [66/100], Step [20/760], Total Loss: 3.0505
Epoch [66/100], Step [30/760], Total Loss: 3.0204
Epoch [66/100], Step [40/760], Total Loss: 3.0832
Epoch [66/100], Step [50/760], Total Loss: 3.0391
Epoch [66/100], Step [60/760], Total Loss: 3.0300
Epoch [66/100], Step [70/760], Total Loss: 3.0416
Epoch [66/100], Step [80/760], Total Loss: 3.0542
Epoch [66/100], Step [90/760], Total Loss: 3.0499
Epoch [66/100], Step [100/760], Total Loss: 3.0406
Epoch [66/100], Step [110/760], Total Loss: 3.0402
Epoch [66/100], Step [120/760], Total Loss: 3.0890
Epoch [66/100], Step [130/760], Total Loss: 3.0531
Epoch [66/100], Step [140/760], Total Loss: 3.0783
Epoch [66/100], Step [150/760], Total Loss: 3.0585
Epoch [66/100], Step [160/760], Total Loss: 3.0827
Epoch [66/100], Step [170/760], Total Loss: 3.0811
Epoch [66/100], Step [180/760], Total Loss: 3.0632
Epoch [66/100], Step [190/760], Total Lo

 66%|██████▌   | 66/100 [2:34:01<1:19:18, 139.96s/it]

Epoch [66/100], Step [760/760], Total Loss: 3.1662
Epoch [67/100], Step [10/760], Total Loss: 3.0788
Epoch [67/100], Step [20/760], Total Loss: 3.1376
Epoch [67/100], Step [30/760], Total Loss: 3.0906
Epoch [67/100], Step [40/760], Total Loss: 3.0759
Epoch [67/100], Step [50/760], Total Loss: 3.0299
Epoch [67/100], Step [60/760], Total Loss: 3.0796
Epoch [67/100], Step [70/760], Total Loss: 3.0966
Epoch [67/100], Step [80/760], Total Loss: 3.0869
Epoch [67/100], Step [90/760], Total Loss: 3.1422
Epoch [67/100], Step [100/760], Total Loss: 3.0462
Epoch [67/100], Step [110/760], Total Loss: 3.0492
Epoch [67/100], Step [120/760], Total Loss: 3.1154
Epoch [67/100], Step [130/760], Total Loss: 3.1316
Epoch [67/100], Step [140/760], Total Loss: 3.0823
Epoch [67/100], Step [150/760], Total Loss: 3.0126
Epoch [67/100], Step [160/760], Total Loss: 3.1050
Epoch [67/100], Step [170/760], Total Loss: 3.0847
Epoch [67/100], Step [180/760], Total Loss: 3.0837
Epoch [67/100], Step [190/760], Total Lo

 67%|██████▋   | 67/100 [2:36:21<1:16:58, 139.95s/it]

Epoch [67/100], Step [760/760], Total Loss: 3.1071
Epoch [68/100], Step [10/760], Total Loss: 3.0496
Epoch [68/100], Step [20/760], Total Loss: 3.0242
Epoch [68/100], Step [30/760], Total Loss: 3.0558
Epoch [68/100], Step [40/760], Total Loss: 3.0760
Epoch [68/100], Step [50/760], Total Loss: 3.0762
Epoch [68/100], Step [60/760], Total Loss: 3.1048
Epoch [68/100], Step [70/760], Total Loss: 3.0321
Epoch [68/100], Step [80/760], Total Loss: 3.0559
Epoch [68/100], Step [90/760], Total Loss: 3.0893
Epoch [68/100], Step [100/760], Total Loss: 3.1257
Epoch [68/100], Step [110/760], Total Loss: 3.0538
Epoch [68/100], Step [120/760], Total Loss: 3.0936
Epoch [68/100], Step [130/760], Total Loss: 3.0933
Epoch [68/100], Step [140/760], Total Loss: 3.0207
Epoch [68/100], Step [150/760], Total Loss: 3.0884
Epoch [68/100], Step [160/760], Total Loss: 3.0782
Epoch [68/100], Step [170/760], Total Loss: 3.0832
Epoch [68/100], Step [180/760], Total Loss: 3.0968
Epoch [68/100], Step [190/760], Total Lo

In [23]:
model(Xtr[:100])

(tensor([[8.3641e-04, 1.3017e-09, 9.9916e-01],
         [7.9041e-08, 5.2794e-04, 9.9947e-01],
         [1.0000e+00, 1.1581e-09, 1.7903e-11],
         [4.9460e-10, 9.5416e-25, 1.0000e+00],
         [2.8009e-15, 0.0000e+00, 1.0000e+00],
         [9.8073e-01, 0.0000e+00, 1.9271e-02],
         [4.2647e-22, 0.0000e+00, 1.0000e+00],
         [1.7237e-05, 9.9989e-01, 8.8235e-05],
         [2.1224e-08, 1.0000e+00, 1.7411e-19],
         [1.0000e+00, 0.0000e+00, 1.0993e-08],
         [9.9997e-01, 0.0000e+00, 3.0800e-05],
         [4.1415e-01, 5.8585e-01, 1.4535e-17],
         [1.4560e-13, 9.9967e-01, 3.2696e-04],
         [9.9990e-01, 1.0382e-04, 8.5768e-12],
         [2.6331e-07, 1.0000e+00, 1.6438e-17],
         [9.9954e-01, 2.0068e-06, 4.6026e-04],
         [1.0000e+00, 5.2905e-19, 8.8479e-17],
         [1.0686e-05, 1.4292e-02, 9.8570e-01],
         [1.0000e+00, 4.3145e-12, 1.7805e-17],
         [1.0000e+00, 4.2292e-06, 3.8462e-18],
         [2.8464e-02, 9.7154e-01, 2.6349e-21],
         [1.6

In [21]:
y1[:100]

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

# Training loop
num_epochs = 100
for epoch in tqdm(range(num_epochs)):
    for batch_idx, (data, target1, target2, target3) in enumerate(dataloader):
        # Move data and targets to CUDA device
        data, target1, target2, target3 = data.cuda(), target1.cuda(), target2.cuda(), target3.cuda()

        # Forward pass
        outputs1, outputs2, outputs3 = model(data)
        
        # Compute the loss for both tasks
        loss1 = criterion(outputs1, target1)
        loss2 = criterion(outputs2, target2)
        loss3 = criterion(outputs3, target3)
        total_loss = loss1 + loss2 + loss3
        
        # Backward and optimize
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        # Print progress
        if (batch_idx+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(dataloader)}], Total Loss: {total_loss.item():.4f}')

In [ ]:
class MyConv1dPadSame(nn.Module):
    """
    extend nn.Conv1d to support SAME padding
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super(MyConv1dPadSame, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.conv = torch.nn.Conv1d(
            in_channels=self.in_channels, 
            out_channels=self.out_channels, 
            kernel_size=self.kernel_size, 
            stride=self.stride, 
            groups=self.groups)

    def forward(self, x):
        
        net = x
        
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.conv(net)

        return net

In [ ]:
class MyMaxPool1dPadSame(nn.Module):
    """
    extend nn.MaxPool1d to support SAME padding
    """
    def __init__(self, kernel_size):
        super(MyMaxPool1dPadSame, self).__init__()
        self.kernel_size = kernel_size
        self.stride = 1
        self.max_pool = torch.nn.MaxPool1d(kernel_size=self.kernel_size)

    def forward(self, x):
        
        net = x
        
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.max_pool(net)
        
        return net

In [ ]:
class BasicBlock(nn.Module):
    """
    ResNet Basic Block
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups, downsample, use_bn, use_do, is_first_block=False):
        super(BasicBlock, self).__init__()
        
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.stride = stride
        self.groups = groups
        self.downsample = downsample
        if self.downsample:
            self.stride = stride
        else:
            self.stride = 1
        self.is_first_block = is_first_block
        self.use_bn = use_bn
        self.use_do = use_do

        # the first conv
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.relu1 = nn.ReLU()
        self.do1 = nn.Dropout(p=0.5)
        self.conv1 = MyConv1dPadSame(
            in_channels=in_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=self.stride,
            groups=self.groups)

        # the second conv
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.do2 = nn.Dropout(p=0.5)
        self.conv2 = MyConv1dPadSame(
            in_channels=out_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=1,
            groups=self.groups)
                
        self.max_pool = MyMaxPool1dPadSame(kernel_size=self.stride)

    def forward(self, x):
        
        identity = x
        
        # the first conv
        out = x
        if not self.is_first_block:
            if self.use_bn:
                out = self.bn1(out)
            out = self.relu1(out)
            if self.use_do:
                out = self.do1(out)
        out = self.conv1(out)
        
        # the second conv
        if self.use_bn:
            out = self.bn2(out)
        out = self.relu2(out)
        if self.use_do:
            out = self.do2(out)
        out = self.conv2(out)
        
        # if downsample, also downsample identity
        if self.downsample:
            identity = self.max_pool(identity)
            
        # if expand channel, also pad zeros to identity
        if self.out_channels != self.in_channels:
            identity = identity.transpose(-1,-2)
            ch1 = (self.out_channels-self.in_channels)//2
            ch2 = self.out_channels-self.in_channels-ch1
            identity = F.pad(identity, (ch1, ch2), "constant", 0)
            identity = identity.transpose(-1,-2)
        
        # shortcut
        out += identity

        return out

In [ ]:
class ResNet1D(nn.Module):
    """
    
    Input:
        X: (n_samples, n_channel, n_length)
        Y: (n_samples)
        
    Output:
        out: (n_samples)
        
    Pararmetes:
        in_channels: dim of input, the same as n_channel
        base_filters: number of filters in the first several Conv layer, it will double at every 4 layers
        kernel_size: width of kernel
        stride: stride of kernel moving
        groups: set largest to 1 as ResNeXt
        n_block: number of blocks
        n_classes: number of classes
        
    """

    def __init__(self, in_channels, base_filters, kernel_size, stride, groups, n_block, n_classes, downsample_gap=2, increasefilter_gap=4, use_bn=True, use_do=True, verbose=False):
        super(ResNet1D, self).__init__()
        
        self.verbose = verbose
        self.n_block = n_block
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.use_bn = use_bn
        self.use_do = use_do

        self.downsample_gap = downsample_gap # 2 for base model
        self.increasefilter_gap = increasefilter_gap # 4 for base model

        # first block
        self.first_block_conv = MyConv1dPadSame(in_channels=in_channels, out_channels=base_filters, kernel_size=self.kernel_size, stride=1)
        self.first_block_bn = nn.BatchNorm1d(base_filters)
        self.first_block_relu = nn.ReLU()
        out_channels = base_filters
                
        # residual blocks
        self.basicblock_list = nn.ModuleList()
        for i_block in range(self.n_block):
            # is_first_block
            if i_block == 0:
                is_first_block = True
            else:
                is_first_block = False
            # downsample at every self.downsample_gap blocks
            if i_block % self.downsample_gap == 1:
                downsample = True
            else:
                downsample = False
            # in_channels and out_channels
            if is_first_block:
                in_channels = base_filters
                out_channels = in_channels
            else:
                # increase filters at every self.increasefilter_gap blocks
                in_channels = int(base_filters*2**((i_block-1)//self.increasefilter_gap))
                if (i_block % self.increasefilter_gap == 0) and (i_block != 0):
                    out_channels = in_channels * 2
                else:
                    out_channels = in_channels
            
            tmp_block = BasicBlock(
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=self.kernel_size, 
                stride = self.stride, 
                groups = self.groups, 
                downsample=downsample, 
                use_bn = self.use_bn, 
                use_do = self.use_do, 
                is_first_block=is_first_block)
            self.basicblock_list.append(tmp_block)

        # final prediction
        self.final_bn = nn.BatchNorm1d(out_channels)
        self.final_relu = nn.ReLU(inplace=True)
        # self.do = nn.Dropout(p=0.5)
        self.dense = nn.Linear(out_channels, n_classes)
        # self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        out = x
        
        # first conv
        if self.verbose:
            print('input shape', out.shape)
        out = self.first_block_conv(out)
        if self.verbose:
            print('after first conv', out.shape)
        if self.use_bn:
            out = self.first_block_bn(out)
        out = self.first_block_relu(out)
        
        # residual blocks, every block has two conv
        for i_block in range(self.n_block):
            net = self.basicblock_list[i_block]
            if self.verbose:
                print('i_block: {0}, in_channels: {1}, out_channels: {2}, downsample: {3}'.format(i_block, net.in_channels, net.out_channels, net.downsample))
            out = net(out)
            if self.verbose:
                print(out.shape)

        # final prediction
        if self.use_bn:
            out = self.final_bn(out)
        out = self.final_relu(out)
        out = out.mean(-1)
        if self.verbose:
            print('final pooling', out.shape)
        # out = self.do(out)
        out = self.dense(out)
        if self.verbose:
            print('dense', out.shape)
        # out = self.softmax(out)
        if self.verbose:
            print('softmax', out.shape)
        
        return out    

In [ ]:
resnet = ResNet1D(1, 16, 3, 1, 1, 2, 10, verbose=True)